# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Increase fidelity
    - Assess difference in speed of calculation and optimal values
- Loop over wind direction
    - Parallelise using PQDM
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [ ]:
# import libraries
import logging

import numpy as np
import xarray as xr

import utils

logging.basicConfig(level=logging.INFO)

## Baseline values

In [ ]:
# run baseline simulation
sim_res_ref, Sector_frequency, P = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=utils.WS_DEFAULT,
    wd=utils.WD_DEFAULT,
)

## Optimise across for a single wind direction

In [ ]:
# define constants
wd = 270
yaw_shape = (len(sim_res_ref.wt), 1, len(sim_res_ref.ws))

In [ ]:
# initialise optimal yaw dataset
yaw_opt = xr.Dataset(
    coords={
        "wt": list(sim_res_ref.wt.values),
        "wd": [wd],
        "ws": list(sim_res_ref.ws.values),
    },
)

In [ ]:
# run optimisation and save output
yaw_opt_power, yaw_opt_lcoe = utils.optimise_direction(
    wd=wd,
    sim_res_ref=sim_res_ref,
    Sector_frequency=Sector_frequency,
    P=P,
)
yaw_opt["power"] = (["wt", "wd", "ws"], yaw_opt_power)
yaw_opt["lcoe"] = (["wt", "wd", "ws"], yaw_opt_lcoe)

In [ ]:
# rerun simulation for optimum
sim_res_opt, _, _ = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=yaw_opt.lcoe.values.reshape(yaw_shape),
    ws=utils.WS_DEFAULT,
    wd=[wd],
    sim_res_ref=sim_res_ref.sel(wd=[wd]),
    Sector_frequency=Sector_frequency,
    P=P,
)
sim_res_opt

In [ ]:
# display comaprison of optimum to baseline
print(
    f"Baseline LCoE [USD/MWh]      : {sim_res_ref.lcoe_direction.sel(wd=wd).values:.3f}"
)
print(f"Optimum LCoE [USD/MWh]       : {sim_res_opt.lcoe_overall.values:.3f}")
print(
    f"Baseline Capacity Factor [%] : {100*sim_res_ref.cap_fac_direction.sel(wd=wd).values:.3f}"
)
print(f"Optimum Capacity Factor [%]  : {100*sim_res_opt.cap_fac_overall.values:.3f}")